In [1]:
import tensorly as tl
from utils import *
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy.solvers.inequalities import *
from sympy.polys import Poly
from sympy.abc import x
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed

In [2]:
#generates random rank 3 tensors
def rank_tree(size = 100):
    return (low_tensor(size) + low_tensor(size) + low_tensor(size)) 

# random tensor of some dim
def rand_tensor(size = 100000):
    return tl.tensor(np.random.randint(1, size, size=(3,2,2)))*1.0

# generates rank 1 tensors
def low_tensor(size = 100):
    a = np.random.randint(1, size, size=3) 
    b = np.random.randint(1, size, size=2)
    c = np.random.randint(1, size, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

In [3]:

def proc_tensor(t):
    tens = t.copy()
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    if abs(M_a.det()) < 10:
        print("warning det too small")
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    sol = la.solve(np.array(M.T@M,dtype = "float"), np.array(M.T@R, dtype = "float"))
    ret = np.append(np.array([b], dtype = "float"),sol)
    return np.all(ret >=0)


def test(tens):
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_simple(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    r4 = check_r4(tens)
    if r4  and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

def test_r2(tens):
    A = Matrix(tens[0]) * Matrix(tens[1]).inv()
    res1 = Matrix(A.eigenvects()[0][2])
    res2 = Matrix(A.eigenvects()[1][2])

    B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
    res3 = Matrix(B.eigenvects()[0][2])
    res4 = Matrix(B.eigenvects()[1][2])
    
    T1 = np.kron(res1,res3) * 1.0
    T2 = np.kron(res2,res4) * 1.0
    assert(np.all(T1 >= 0) or np.all(T1 <= 0))
    assert(np.all(T2 >= 0) or np.all(T2 <= 0))
    T1 = Matrix(abs(T1))
    T2 = Matrix(abs(T2))

    a = symbols('a')
    M = Matrix([[T1,T2]])
    A1 = Matrix([a,a,1,1])
    A2 = Matrix([a,1,a,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])

    a1 = solve(M1.det())[0]
    A1 = Matrix([a1,a1,1,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M1_sub = M1[0:3,0:3]
    R = M1[0:3,3]
    d1 = False
    if abs(M1_sub.det()) > 0:
        c1 = np.array(M1_sub.inv() @ R, dtype = "float")
        d1 = np.all(c1 >=0)

    a2 = solve(M2.det())[0]
    A2 = Matrix([a2,1,a2,1])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
    M2_sub = M2[0:3,0:3]
    R = M2[0:3,3]
    d2 = False
    if abs(M2_sub.det()) > 0:
        c2 = np.array(M2_sub.inv() @ R, dtype = "float")
        d2 = np.all(c2 >=0)
    return d1 or d2

def check_comb(tens):
    t = tens.copy()
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[1],t[2]]))
    a1 = False
    a2 = False
    a3 = False
    if a:
        a1 = test_r2(t)
        if a1:
            return a1
    if b:
        a2 = test_r2(tl.tensor([t[0],t[2],t[1]]))
        if a2:
            return a2
    if c:
        a3 = test_r2(tl.tensor([t[1],t[2],t[0]]))
        if a3:
            return a3
    return a1 or a2 or a3

In [4]:

# assumes the nonnegative rank 2 subtensor are the first two slices
def test_simple(tens, M, A1, tol = 0.0001):
    a,b = symbols('a,b')
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    a1 = solve(M1.det(),a)[0]
    A1 = A1.subs(a,a1)
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M1_sub = M1[1:4,0:3]
    R = M1[1:4,3]
    res1 = M1_sub.inv() @ R
    res1[0] = fraction(simplify(res1[0]))[0] * fraction(simplify(res1[0]))[1]
    res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
    res1[2] = fraction(simplify(res1[2]))[0] * fraction(simplify(res1[2]))[1]
    a2 = fraction(simplify(a1))[0] * fraction(simplify(a1))[1]
    s1 = solveset(a2 >= 0, b, S.Reals)
    s2 = solveset(res1[0] >= 0, b, S.Reals)
    s3 = solveset(res1[1] >=0,b,S.Reals)
    s4 = solveset(res1[2] >=0,b,S.Reals)
    s5 = solveset(b >=0,b,S.Reals)
    sol = Intersection(s1,s2,s3,s4,s5)
    return sol.measure > tol


# initializes the matrix with the appropriate rank 1 terms given the tensors has nonneg rank 2 subtensor
def init_mat(tens):
    A = tens[0] @ la.inv(tens[1])
    eig = la.eig(A)
    B = tens[0].T @ la.inv(tens[1]).T
    eig1 = la.eig(B)

    a1 = eig[1][:,0]
    b1 = eig[1][:,1]

    a2 = eig1[1][:,0]
    b2 = eig1[1][:,1]

    T1 = np.kron(a1,b2)
    T2 = np.kron(b1,a2)
    #ret = Matrix([[T1,T2]])
    if eig[0][0] == eig[0][1]:
        print('identical eigenvalue, result should be discarded')
    
    # take the smaller tensor and check which decomposition to use
    temp = tl.tensor([tens[0],tens[1]])
    if mat_comb_small(temp,T1,T2):
        T1 = Matrix(abs(T1))  
        T2 = Matrix(abs(T2))
        ret = Matrix([[T1,T2]])
        #print(T1)
        
    T1 = np.kron(a1,a2)
    T2 = np.kron(b1,b2)
    if mat_comb_small(temp,T1,T2):
        T1 = Matrix(abs(T1))
        T2 = Matrix(abs(T2))
        ret = Matrix([[T1,T2]])
        #print(T1)
    return ret

# as before, checks if the matrix decompostion is valid by reconstructing it
def mat_comb_small(tens,K1,K2):
    T1 = Matrix(K1.reshape(4))
    T2 = Matrix(K2.reshape(4))
    pos1 = abs(sum(T1) / sum(abs(T1)))
    pos2 = abs(sum(T2) / sum(abs(T2)))
    if pos1 + pos2 != 2:
        print('returning')
        return False
    M = abs(Matrix([[T1,T2]]))
    R1 = tens[0].reshape(4)
    R2 = tens[1].reshape(4)
    sol1 = la.solve(np.array(M.T @ M, dtype = "float"), np.array(M.T @ R1, dtype = "float"))
    sol2 = la.solve(np.array(M.T @ M, dtype = "float"), np.array(M.T @ R2, dtype = "float"))
    a3 = [sol1[0],sol2[0]]
    b3 = [sol1[1],sol2[1]]
    m1 = np.kron(a3,abs(T1).reshape(1,4)).reshape(2,2,2)
    m2 = np.kron(b3,abs(T2).reshape(1,4)).reshape(2,2,2)
    #print(np.max(abs(m1+m2-tens)))
    return (np.max(abs(m1+m2-tens))<0.1)


    
# checks if a given 2x2x3 has nonneg rank 2 subtensor and if it does check if the third slice is a linear combination
def check_simple(tens):
    t = tens.copy()
    c1 = check_r2(tl.tensor([t[0],t[1]]))
    c2 = check_r2(tl.tensor([t[0],t[2]]))
    c3 = check_r2(tl.tensor([t[1],t[2]]))
    a1 = False
    a2 = False
    a3 = False
    if c1:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = t
        M = init_mat(temp)
        a1 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if a1:
            return a1
    if c2:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = tl.tensor([t[0],t[2],t[1]])
        M = init_mat(temp)
        a2 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if a2:
            return a2
    if c3:
        a,b = symbols('a,b')
        A1 = Matrix([a*b,a,b,1])
        A2 = Matrix([a,a*b,1,b])
        A3 = Matrix([b,1,a*b,a])
        A4 = Matrix([1,b,a,a*b])
        temp = tl.tensor([t[1],t[2],t[0]])
        M = init_mat(temp)
        a3 = test_simple(temp,M,A1) or test_simple(temp,M,A2) or test_simple(temp,M,A3) or test_simple(temp,M,A4)
        if a3:
            return a3
    return False

def loop_rotations(i):
    tens = rand_tensor()
    tens = rank_tree(size = 50)
    tens = tensors[i]
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_simple(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 1)
    r4 = check_r4(tens)
    if r4:
        ret[4] = 1
    if r4 and r2:
        ret[3] = 1
    if r2 and not r4:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    tensors[i] = (tensors[i],ret[0])
    return ret

def loop_rotations_old(i):
    tens = rand_tensor()
    tens = rank_tree(size = 50)
    tens = tensors[i]
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_comb(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    r4 = check_r4(tens)
    if r4:
        ret[4] = 1
    if r4 and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

In [15]:
total = 10000
tensors = []
for i in range(total):
    tensors.append(rand_tensor())

In [ ]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations_old)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[904, 797, 440, 0, 0, 0, 544, 64, 107, 0, 0]
124.45250034332275


In [50]:
te = time.time()
total = 2000
results = Parallel(n_jobs=6)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[318, 282, 108, 86, 1010, 0, 187, 63, 36, 0, 0]
272.9294993877411


In [16]:
results = []
res = []
te = time.time()
for i in range(1000):
    tens = tensors[i]
    ret = [0] *11
    if proc_tensor(tens) or proc_tensor(mat_trans(tens)) or proc_tensor(mat_inv(tens)) or proc_tensor(rotate(tens)):
        ret[1] = 1
        ret[0] = 1
    if check_simple(tens):
        ret[2] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 1)
    r4 = check_r4(tens)
    if r4:
        ret[4] = 1
    if r4 and r2:
        ret[3] = 1
    if not r4 and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    res.append((tensors[i],max(ret[0],ret[4], ret[0])))
    results.append(ret)
print(time.time() - te)
print([sum(x) for x in zip(*results)])

516.8704991340637
[191, 168, 60, 50, 480, 0, 101, 34, 23, 0, 0]


[173, 150, 54, 41, 511, 0, 99, 34, 23, 0, 0]


In [18]:
wrongs = []
for tens in res:
    if tens[1] != 1:
        if check_rank(tens[0],3,n=20):
            wrongs.append((tens[0],3))
        else:
            wrongs.append((tens[0],4))

In [20]:
counter = 0
for tens in wrongs:
    if tens[1] == 3:
        counter +=1
print(counter)

20


In [17]:
a,b = symbols('a,b')
test_simple(tens,init_mat(tens),A1 = Matrix([a*b,a,b,1]))

True

In [26]:
A1 = Matrix([a*b,a,b,1])
M = init_mat(tens)

In [27]:
a,b = symbols('a,b')
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
a1 = solve(M1.det(),a)[0]
A1 = A1.subs(a,a1)
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[1:4,0:3]
R = M1[1:4,3]
res1 = M1_sub.inv() @ R
res1

Matrix([
[103064.0*(-14386891197102.1*b**2 + 39540403745329.5*b - 10247916134103.9)/(90291819975.1211*b**2 - 330451987880.208*b + 88158528685.7426) + 105354.0*(-0.0001678466796875*b**2 - 105345726657.839*b + 30520333847.716)/(90291819975.1211*b**2 - 330451987880.208*b + 88158528685.7426) + 51228.0*(29347443300247.1*b**2 - 80595178439341.3*b + 20886419229356.7)/(90291819975.1211*b**2 - 330451987880.208*b + 88158528685.7426)],
[    51228.0*(-10097075262131.1*b**2 + 58831942339144.5*b - 83591193326143.0)/(105398020680.906*b**2 - 385737993344.707*b + 102907820799.031) + 105354.0*(6.103515625e-5*b**2 - 85963970050.8978*b + 289707697125.805)/(105398020680.906*b**2 - 385737993344.707*b + 102907820799.031) + 103064.0*(5072061240561.94*b**2 - 29263326889482.7*b + 41013997150452.1)/(105398020680.906*b**2 - 385737993344.707*b + 102907820799.031)],
[                                                                           51228.0*(97576351051047.9 - 39721277835901.0*b)/(156071368284.147*b**2 - 57

In [28]:

    res1[0] = fraction(simplify(res1[0]))[0] * fraction(simplify(res1[0]))[1]
    res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
    res1[2] = fraction(simplify(res1[2]))[0] * fraction(simplify(res1[2]))[1]
    a2 = fraction(simplify(a1))[0] * fraction(simplify(a1))[1]
    s1 = solveset(a2 >= 0, b, S.Reals)
    s2 = solveset(res1[0] >= 0, b, S.Reals)
    s3 = solveset(res1[1] >=0,b,S.Reals)
    s4 = solveset(res1[2] >=0,b,S.Reals)
    s5 = solveset(b >=0,b,S.Reals)
    sol = Intersection(s1,s2,s3,s4,s5)

In [29]:
sol

Union({0.289715917446054, 3.37010606832476}, Interval(2.45652598222448, 2.84184291825452))

In [234]:
T1 = Matrix(A1)
T2 = Matrix(B1)
pos1 = abs(sum(T1) / sum(abs(T1)))
pos2 = abs(sum(T2) / sum(abs(T2)))
Matrix([[T1,T2]])

Matrix([
[-0.665354005492337, -0.647215077985079],
[-0.700768373170271, -0.244624596891531],
[-0.177195317059848, -0.675360927647418],
[ -0.18662677769188, -0.255262740782205]])

In [205]:
a,b = symbols('a,b')
M = abs(Matrix([[T1,T2]]))
A1 = Matrix([a*b,a,b,1])
A2 = Matrix([a,a*b,1,b])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
M1

Matrix([
[0.665354005492337, 0.647215077985079, a*b, 132375.0],
[0.700768373170271, 0.244624596891531,   a,  71016.0],
[0.177195317059848, 0.675360927647418,   b, 119559.0],
[ 0.18662677769188, 0.255262740782205,   1,  50472.0]])

In [206]:
a1 = solve(M1.det(),a)[0]
a1
x1 = solve(M2.det(),a)[0]
a1

4.0*(131046904946624.0*b - 405421457728585.0)/(32016249831265.0*b - 329735984781862.0)

In [207]:
A1 = Matrix([a1 *b,a1,b,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[1:4,0:3]
R = M1[1:4,3]
res = M1_sub.inv() @ R
res

Matrix([
[50472.0*(-1653531670876.45*b**2 + 91771739691246.7*b - 231230217597509.0)/(900540384875.792*b**2 - 3237633312183.22*b + 2262194694390.39) + 119559.0*(6.103515625e-5*b**2 - 26596316753998.9*b + 70367156209103.2)/(900540384875.792*b**2 - 3237633312183.22*b + 2262194694390.39) + 71016.0*(1725439843914.22*b**2 - 22335418082982.7*b + 47015841205106.0)/(900540384875.792*b**2 - 3237633312183.22*b + 2262194694390.39)],
[ 71016.0*(-1800163115506.2*b**2 + 20249104430078.7*b - 17602973147768.9)/(1285075667444.51*b**2 - 4620119052371.32*b + 3228163229108.41) + 119559.0*(-0.000244140625*b**2 + 22713792747988.3*b - 21565917589041.5)/(1285075667444.51*b**2 - 4620119052371.32*b + 3228163229108.41) + 50472.0*(6759466103932.49*b**2 - 97599658004624.7*b + 86573784643454.8)/(1285075667444.51*b**2 - 4620119052371.32*b + 3228163229108.41)],
[                                                                        50472.0*(99342202942211.1 - 9645792194291.97*b)/(2989067705852.98*b**2 - 1074633113558

In [208]:
res1 = res.copy()
res1[0] = fraction(simplify(res1[0]))[0] * fraction(simplify(res1[0]))[1]
res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
res1[2] = fraction(simplify(res1[2]))[0] * fraction(simplify(res1[2]))[1]
a2 = fraction(simplify(a1))[0] * fraction(simplify(a1))[1]
res1[2]

(7.39794841314304e+15 - 718315790709632.0*b)*(2989067705852.98*b**2 - 10746331135583.1*b + 7508661709032.52)

In [209]:
s1 = solveset(a2 >= 0, b, S.Reals)
s2 = solveset(res1[0] >= 0, b, S.Reals)
s3 = solveset(res1[1] >=0,b,S.Reals)
s4 = solveset(res1[2] >=0,b,S.Reals)
s5 = solveset(b >=0,b,S.Reals)

In [210]:
sol = Intersection(s1,s2,s3,s4,s5)
sol

Union(Interval(0, 0.785901550908152), Interval(2.64574816355059, 2.64574816355059), Interval(2.67133583965050, 3.09371257485032))

In [30]:
b1 = 2.5
a3 = a1.subs(b,b1)
A1 = Matrix([a3 *b1,a3,b1,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[1:4,0:3]
R = M1[1:4,3]
print(a3)
sol = M1_sub.inv() @ R
sol

0.672727272740676


Matrix([
[89809.3139622101],
[90318.8060789435],
[2339.68976196833]])

In [32]:
M[:,0]

Matrix([
[0.430333194122095],
[0.127691290837016],
[0.856674721136186],
[0.254198148001299]])

In [33]:
m1 = sol[0] * M[:,0]
m2 = sol[1] * M[:,1]
m3 = sol[2] * np.kron([a3,1],[b1,1]).reshape(4,1)
m1+m2+m3

Matrix([
[ 53646.0],
[ 51228.0],
[105354.0],
[103064.0]])

In [34]:
tens

array([[[ 51703.,  31170.],
        [103173.,  62902.]],

       [[ 46665.,  54534.],
        [ 93531., 110750.]],

       [[ 53646.,  51228.],
        [105354., 103064.]]])

In [46]:
sol[0] * M[3,0] + sol[1] * M[3,1] + sol[2] 

76884.9999999998

In [47]:
A2 = Matrix([x1,x1*b,1,b])
M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
M2_sub = M2[1:4,0:3]
R = M2[1:4,3]
res = M2_sub.inv() @ R
res

Matrix([
[76885.0*(-352354435940.077*b**2 + 1002676437237.39*b - 658748975029.015)/(38865806927.4149*b**2 - 108785341530.997*b + 69368983263.2118) + 62140.0*(15762631135.1484*b**2 - 28630467847.2507*b + 0.0001220703125)/(38865806927.4149*b**2 - 108785341530.997*b + 69368983263.2118) + 118904.0*(268732669997.578*b**2 - 771702969359.745*b + 515099228472.964)/(38865806927.4149*b**2 - 108785341530.997*b + 69368983263.2118)],
[                  62140.0*(176499745199.631*b**2 - 173437095050.392*b)/(64846240819.0242*b**2 - 181504541194.099*b + 115739724700.894) + 118904.0*(-530871155189.459*b**2 + 1081880972177.39*b - 550500524116.314)/(64846240819.0242*b**2 - 181504541194.099*b + 115739724700.894) + 76885.0*(696062769165.078*b**2 - 1608077746493.19*b + 908058159601.309)/(64846240819.0242*b**2 - 181504541194.099*b + 115739724700.894)],
[                                                                         62140.0*(1427706690911.84 - 1352908093058.75*b)/(799910420878.048*b**2 - 223894819659

In [48]:
res1 = res.copy()
res1[0] = fraction(simplify(res1[0]))[0] * fraction(simplify(res1[0]))[1]
res1[0] = expand(res1[0])
res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
res1[1] = expand(res1[1])
res1[2] = fraction(simplify(res1[2]))[0] * fraction(simplify(res1[2]))[1]
res1[2] = expand(res1[2])
x2 = fraction(simplify(x1))[0] * fraction(simplify(x1))[1]
s1 = solveset(x2 >= 0, b, S.Reals)
s2 = solveset(res1[0] >= 0, b, S.Reals)
s3 = solveset(res1[1] >=0,b,S.Reals)
s4 = solveset(res1[2] >=0,b,S.Reals)
s5 = solveset(b >= 0,b,S.Reals)
sol = Intersection(s1,s2,s3,s4,s5)
sol

Union(Interval(0.998880933810523, 1.05528727208955), Interval(3.25774269893864, oo))

In [49]:
b1 = 10
a3 = x1.subs(b,b1)
A2 = Matrix([a3,a3*b1,1,b1])
M1 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
M1_sub = M1[1:4,0:3]
R = M1[1:4,3]
print(a3)
sol = M1_sub.inv() @ R
sol

1.30842316219051


Matrix([
[150044.266458094],
[17301.3223127297],
[1255.19541710081]])

In [22]:
sol[0] * M[3,0] + sol[1] * M[3,1] + sol[2] *b1

38022.0000000000

In [534]:
A1 = Matrix([a1 *b,a1,b,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M2_sub = M1[0:3,0:3]
R = M1[0:3,3]
res = M2_sub.adjugate() @ R
res

Matrix([
[-1670.0*(-38235434973269.5*b**2 + 6232566579894.77*b)/(335415311473722.0*b - 735361505608343.0) + 8204.0*(-475872744554206.0*b**2 + 724412708259327.0*b - 105438647461885.0)/(335415311473722.0*b - 735361505608343.0) + 5748.0*(336814697480797.0*b**2 - 831567417226825.0*b + 126600258732122.0)/(335415311473722.0*b - 735361505608343.0)],
[ 1670.0*(245631480973854.0*b**2 - 589294909971145.0*b)/(335415311473722.0*b - 735361505608343.0) - 8204.0*(-180547267144371.0*b**2 + 678565084187433.0*b - 588772728633766.0)/(335415311473722.0*b - 735361505608343.0) - 5748.0*(127788308660508.0*b**2 - 341107606986848.0*b + 82842074462712.2)/(335415311473722.0*b - 735361505608343.0)],
[                                                                                                                                                                                                                                                                                                                            64

In [535]:
res1 = res.copy()
res1[0] = fraction(simplify(res[0]))[0] * fraction(simplify(res[0]))[1]
res1[0] = expand(res1[0])
solve(res1[0],b)

[0.163004986977442 - 0.e-23*I,
 0.442407478548126 + 0.e-20*I,
 2.1923909864978 - 0.e-22*I]

In [536]:
res1[1] = fraction(simplify(res1[1]))[0] * fraction(simplify(res1[1]))[1]
res1[1] = expand(res1[1])
solve(res1[1],b)

[1.5687735388356 + 0.e-22*I,
 2.1923909864982 + 0.e-23*I,
 2.39910172602749 - 0.e-23*I]

In [537]:
solve(res1[2],b)

[]

In [538]:
a2 = fraction(simplify(a1))[0] * fraction(simplify(a1))[1]
print(LC(a2))
solve(a2,b)

1.58951841958048e+29


[1.35927742265699, 2.19239098649781]

In [393]:
b1 = 2
a3= a1.subs(b,b1)
A1 = Matrix([a3 *b1,a3,b1,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[0:3,0:3]
R = M1[0:3,3]
M1_sub.inv() @ R

Matrix([
[20064.5114342207],
[ 16074.895707293],
[ 115.74843886333]])

In [286]:
a,b = symbols('a,b')
A1 = Matrix([a*b,a,b,1])
A2 = Matrix([a,a,b,b])
M = init_mat(tens)
print(test_simple(tens,M,A2))

False


In [14]:
c2 = False
tens = rank_tree()
while not (c2):
    tens = rank_tree(size = 60)
    c2 = check_r2(tl.tensor([tens[0],tens[1]]))
tens    

array([[[100281.,  72018.],
        [165525., 106632.]],

       [[ 47655.,  42213.],
        [ 60747.,  53630.]],

       [[139972., 115281.],
        [181898., 142410.]]])

In [186]:
a,b = symbols('a,b')
A1 = Matrix([a*b,a,b,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
a1 = solve(M1.det(),a)[0]
b1 = 1
a_s = a1.subs(b,b1)
A1 = Matrix([a_s *b1,a_s,b1,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[1:4,0:3]
R = M1[1:4,3]
print(a_s)
sol = M1_sub.inv() @ R
sol

0.419869229207951


Matrix([
[18967.2957060309],
[  18595.99078719],
[2749.34332580756]])

In [176]:
A = Matrix(tens[0]) * Matrix(tens[1]).inv()
#a1
t1 = Matrix(A.eigenvects()[0][2])
#b1
s1 = Matrix(A.eigenvects()[1][2])

B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
#a2
t2 = Matrix(B.eigenvects()[0][2])
#b2
s2 = Matrix(B.eigenvects()[1][2])
B.eigenvects()

[(1.33693794316659,
  1,
  [Matrix([
   [0.960484497657604],
   [0.278333486593725]])]),
 (2.68167700894905,
  1,
  [Matrix([
   [0.0934687310359338],
   [  1.06822700449506]])])]

In [177]:
T1 = np.kron(t1,s2) * 1.0
T2 = np.kron(s1,t2) * 1.0
assert(np.all(T1 >= 0) or np.all(T1 <= 0))
assert(np.all(T2 >= 0) or np.all(T2 <= 0))
T1 = Matrix(abs(T1))
T2 = Matrix(abs(T2))
M = Matrix([[T1,T2]])
M[0:2,0:2]

Matrix([
[0.0729918443053287,  0.153107058100863],
[ 0.834202607981002, 0.0443680469671803]])

In [187]:
R1 = tens[0].reshape(4)
R2 = tens[1].reshape(4)
res1 = M[0:2,0:2].inv() @ R1[0:2]
res2 = M[0:2,0:2].inv() @ R2[0:2]
a3 = [res1[0],res2[0],sol[0]]
b3 = [res1[1],res2[1],sol[1]]
a3

[153611.660450665, 57281.9395990069, 18967.2957060309]

In [188]:
m1 = np.kron(a3,np.kron(t1.T,s2.T)).reshape(3,2,2) 
m2 = np.kron(b3,np.kron(s1.T,t2.T)).reshape(3,2,2)
m3 = np.kron([0,0,sol[2]],np.kron([a_s,1],[b1,1])).reshape(3,2,2)
#(res1[0] * np.kron(t1,s2)).reshape(2,2)+ (res1[1] * np.kron(s1,t2)).reshape(2,2)
m1+m2+m3

array([[[20270.0000000000, 130768.000000000],
        [88612.0000000000, 125576.000000000]],

       [[10956.0000000000, 49748.0000000000],
        [62916.0000000000, 55484.0000000000]],

       [[5386.00000000000, 17802.0000000000],
        [28892.0000000000, 22660.0000000000]]], dtype=object)

In [145]:
np.kron(a3,np.kron(t1.T,t2.T)) + np.kron(b3,np.kron(s1.T,s2.T))

array([[116100.016888476, 43462.1953368047, 99909.0527956453,
        115283.835637823, 43624.4158056263, 19985.4848550710,
        40163.2009613115, 76212.9061706465, -20523.0927220763,
        -8107.09272207622, -17965.4367624648, -24197.4367624649]],
      dtype=object)

array([[116100.016888476, 43462.1953368047],
       [99909.0527956453, 115283.835637823]], dtype=object)

In [146]:
tens

array([[[ 20270., 130768.],
        [ 88612., 125576.]],

       [[ 10956.,  49748.],
        [ 62916.,  55484.]],

       [[  5386.,  17802.],
        [ 28892.,  22660.]]])

In [170]:
# reconstructs the tensor from components
def reconstruct_proc(tens,ret,a,b):
    z = np.array([ret[3],ret[4],ret[5]])
    m1 = np.kron(z,np.kron([0,1],[a,b])).reshape(3,2,2) 
    m2 = np.kron(tens[:,0,0],np.kron([1,ret[1]],[1,0])).reshape(3,2,2)
    m3 = np.kron(tens[:,0,1],np.kron([1,ret[2]],[0,1])).reshape(3,2,2)
    return np.max(m2 + m3 + m1 - tens) < 1

In [25]:
tens[:,0,0]

array([101049.,  95079., 199271.])

In [45]:
t = tens
t

array([[[451862., 264681.],
        [593482., 315984.]],

       [[622864., 492069.],
        [749010., 550118.]],

       [[650396., 226742.],
        [780264., 273968.]]])